In [1]:
using ValkyrieLCMSim
using RigidBodySim
using RigidBodyDynamics
using ValkyrieRobot
using OrdinaryDiffEq
using DiffEqCallbacks
using RigidBodyTreeInspector
using ValkyrieRobot.BipedControlUtil
using RigidBodyDynamics.Contact

In [2]:
val = Valkyrie()
mechanism = val.mechanism
remove_fixed_tree_joints!(mechanism);

In [3]:
# add environment
rootframe = root_frame(mechanism)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, ground);

In [13]:
# Create visualizer
any_open_visualizer_windows() || (new_visualizer_window(); sleep(1));
vis = Visualizer()[:valkyrie]
setgeometry!(vis, mechanism, RigidBodyTreeInspector.parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]))

filename_in_package = "/home/twan/code/RigidBodyDynamics/v0.6/ValkyrieRobot/deps/Valkyrie/urdf/model/meshes/pelvis/pelvis.obj"
ispath


Error encountered while loading "/home/twan/code/RigidBodyDynamics/v0.6/ValkyrieRobot/deps/Valkyrie/urdf/model/meshes/pelvis/pelvis.obj".
Fatal error:


LoadError: [91mArgumentError: input string is empty or only contains whitespace[39m

In [11]:
ValkyrieRobot.packagepath()

"/home/twan/code/RigidBodyDynamics/v0.6/ValkyrieRobot/deps"

In [5]:
feet = Dict(:left => findbody(mechanism, "leftFoot"), :right => findbody(mechanism, "rightFoot"))
hands = Dict(:left => findbody(mechanism, "leftPalm"), :right => findbody(mechanism, "rightPalm"))
robot_info = HumanoidRobotInfo(mechanism, feet, hands);

In [39]:
state = MechanismState(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [40]:
function initialize(state::MechanismState, val::Valkyrie, vis::Visualizer)
    zero!(state)
    kneebend = 1.1
    hipbendextra = 0.1
    for side in instances(Side)
        set_configuration!(state, val.knees[side], [kneebend])
        set_configuration!(state, val.hippitches[side], [-kneebend / 2 + hipbendextra])
        set_configuration!(state, val.anklepitches[side], [-kneebend / 2 - hipbendextra])
    end
    set_configuration!(state, val.basejoint, [1; 0; 0; 0; 0; 0; 1.05]) # TODO
    settransform!(vis, state)
end

initialize (generic function with 1 method)

In [41]:
function periodic_lcm_controller(robot_info::HumanoidRobotInfo{T}, Δt::Number) where T
    lcmcontroller = LCMController(robot_info)
    controller = PeriodicController{T}(num_velocities(robot_info.mechanism), Δt, lcmcontroller)
    initialize_lcmcontroller = (c, t, u, integrator) -> ValkyrieLCMSim.initialize!(lcmcontroller)
    periodic_control_callback = PeriodicCallback(controller, initialize = initialize_lcmcontroller)
    controller, periodic_control_callback
end

periodic_lcm_controller (generic function with 1 method)

In [42]:
controller, periodic_control_callback = periodic_lcm_controller(robot_info, 1 / 300)
initialize(state, val, vis)
final_time = Inf
problem = ODEProblem(state, (0., final_time), controller)
callbacks = CallbackSet(periodic_control_callback, CallbackSet(vis, state))
sol = solve(problem, Tsit5(), abs_tol = 1e-10, dt = 0.05, callback = callbacks);

LoadError: [91mInterruptException:[39m

In [43]:
# animate(vis, state, sol)

In [44]:
# function testfunc(controller, τ, t, state, n)
#     for i = 1 : n
#         controller(τ, t, state)
#     end
# end

# testfunc(controller, τ, t, state, 1)
# Profile.clear_malloc_data()
# @profile testfunc(controller, τ, t, state, 100000)

# using ProfileView
# ProfileView.view()